In [3]:
import pandas as pd
import os

In [4]:
DIR = 'C:/Users/USER/Desktop/python/competition/CarCrashAnalysisAIContest/data/'
os.chdir(DIR)
os.getcwd()
df = pd.read_csv(DIR+'train.csv')
df

,sample_id,video_path,label
0,TRAIN_0000,./train/TRAIN_0000.mp4,7
1,TRAIN_0001,./train/TRAIN_0001.mp4,7
2,TRAIN_0002,./train/TRAIN_0002.mp4,0
3,TRAIN_0003,./train/TRAIN_0003.mp4,0
4,TRAIN_0004,./train/TRAIN_0004.mp4,1
...,...,...,...
2693,TRAIN_2693,./train/TRAIN_2693.mp4,3
2694,TRAIN_2694,./train/TRAIN_2694.mp4,5
2695,TRAIN_2695,./train/TRAIN_2695.mp4,0
2696,TRAIN_2696,./train/TRAIN_2696.mp4,0


In [5]:
df['video_path'] = df['video_path'].apply(lambda x :  './'+'train_thumbnail'+'/'+x.split('/')[2].split('.')[0]+'.jpg')
df

,sample_id,video_path,label
0,TRAIN_0000,./train_thumbnail/TRAIN_0000.jpg,7
1,TRAIN_0001,./train_thumbnail/TRAIN_0001.jpg,7
2,TRAIN_0002,./train_thumbnail/TRAIN_0002.jpg,0
3,TRAIN_0003,./train_thumbnail/TRAIN_0003.jpg,0
4,TRAIN_0004,./train_thumbnail/TRAIN_0004.jpg,1
...,...,...,...
2693,TRAIN_2693,./train_thumbnail/TRAIN_2693.jpg,3
2694,TRAIN_2694,./train_thumbnail/TRAIN_2694.jpg,5
2695,TRAIN_2695,./train_thumbnail/TRAIN_2695.jpg,0
2696,TRAIN_2696,./train_thumbnail/TRAIN_2696.jpg,0


In [15]:
df_timing = df.copy()
def change(x):
    if x == 0:
        return 0
    elif x % 2 == 1:
        return 1
    else:
        return 2

df_timing['label'] = df_timing['label'].apply(change)
def change(x):
    if x == 0:
        return 'Nothing'
    elif x == 1:
        return 'Day'
    else:
        return 'Night'
df_timing['label_name'] = df_timing['label'].apply(change)
df_timing[['label','label_name']].value_counts()

label  label_name
0      Nothing       1783
1      Day            808
2      Night          107
dtype: int64

In [37]:
label2id,id2label = dict(),dict()
for i in df_timing[['label','label_name']].value_counts().items():
    label2id[i[0][1]] = i[0][0]
    id2label[i[0][0]] = i[0][1]
id2label

{0: 'Nothing', 1: 'Day', 2: 'Night'}

In [13]:
df_timing

,sample_id,video_path,label,label_name
0,TRAIN_0000,./train_thumbnail/TRAIN_0000.jpg,1,Day
1,TRAIN_0001,./train_thumbnail/TRAIN_0001.jpg,1,Day
2,TRAIN_0002,./train_thumbnail/TRAIN_0002.jpg,0,Nothing
3,TRAIN_0003,./train_thumbnail/TRAIN_0003.jpg,0,Nothing
4,TRAIN_0004,./train_thumbnail/TRAIN_0004.jpg,1,Day
...,...,...,...,...
2693,TRAIN_2693,./train_thumbnail/TRAIN_2693.jpg,1,Day
2694,TRAIN_2694,./train_thumbnail/TRAIN_2694.jpg,1,Day
2695,TRAIN_2695,./train_thumbnail/TRAIN_2695.jpg,0,Nothing
2696,TRAIN_2696,./train_thumbnail/TRAIN_2696.jpg,0,Nothing


In [1]:
from transformers import SwinConfig, AutoImageProcessor, SwinForImageClassification, TrainingArguments, Trainer
# from datasets import load_dataset, load_metric

import torch
import torchvision
import numpy as np

In [38]:
configuration = SwinConfig()
configuration.num_labels = 10

feature_extractor = AutoImageProcessor.from_pretrained("microsoft/swin-tiny-patch4-window7-224",
                                                        label2id=label2id,
                                                        id2label=id2label,
                                                        ignore_mismatched_sizes = True, 
                                                       )
model = SwinForImageClassification(configuration)

Downloading: 100%|██████████| 255/255 [00:00<00:00, 127kB/s]
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [39]:

from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
train_transforms = Compose(
        [
            RandomResizedCrop((feature_extractor.size['height'], feature_extractor.size['width'])),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

val_transforms = Compose(
        [
            Resize((feature_extractor.size['height'], feature_extractor.size['width'])),
            CenterCrop((feature_extractor.size['height'], feature_extractor.size['width'])),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

In [ ]:
from sklearn.model_selection import train_test_split
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2)


In [41]:
splits = df_timing.train_test_split(test_size=0.1)
train_ds = splits['train']
val_ds = splits['test']

train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

AttributeError: 'DataFrame' object has no attribute 'train_test_split'